In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

C:\Users\Jinbao\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_data = pd.read_csv('dataTrain.csv')
test_data = pd.read_csv('dataA.csv')
submission = pd.read_csv('submit_example_A.csv')
data_nolabel = pd.read_csv('dataNoLabel.csv')

`train.csv`:包含全量数据集的70%（dataNoLabel是训练集的一部分，选手可以自己决定是否使用）

`test.csv`:包含全量数据集的30%

位置类特特征：基于联通基站产生的用户信令数据；`f1~f6`

互联网类特征：基于联通用户上网产生的上网行为数据； `f7~f42`

通话类特征：基于联通用户日常通话、短信产生的数据`f43~f46`

In [3]:
train_data.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,label
0,81167,0,1,mid,0,0,0,153,0,0,...,0,0,0,0,0,0,0,624,1539,0
1,50408,1,1,mid,0,0,21,0,0,0,...,0,0,0,0,0,0,0,186,366,0
2,9114,0,0,high,36,36,120,0,0,0,...,0,0,0,0,0,0,0,24,48,1
3,53228,1,1,low,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,9,0
4,56280,1,1,mid,9,51,294,0,0,0,...,0,0,0,0,0,0,0,42,141,0


In [4]:
print(f'train_data.shape = {train_data.shape}\ntest_data.shape  = {test_data.shape}')

train_data.shape = (59872, 48)
test_data.shape  = (49858, 47)


In [5]:
cat_columns = ['f%d' %idx for idx in range(1, 47)]
data = pd.concat([train_data, test_data])

for col in cat_columns:
    lb = LabelEncoder()
    lb.fit(data[col])
    train_data[col] = lb.transform(train_data[col])
    test_data[col] = lb.transform(test_data[col])
    

In [6]:
for col in cat_columns:
    agg = train_data.groupby([col]).label.agg(['sum', 'count', 'mean'])
    agg.columns = [f'{col}_{agg_col}' for agg_col in agg.columns]
    train_data = train_data.merge(agg, how='left', on=[col])
    test_data = test_data.merge(agg, how='left', on=[col])

In [8]:
num_columns = [col for col in train_data.columns if col not in cat_columns and col not in ['id', 'label']]

In [17]:
test_data = test_data.fillna(0)

In [18]:
feature_columns = cat_columns + num_columns

In [19]:
train = train_data[feature_columns]
label = train_data['label']
test = test_data[feature_columns]

In [21]:
def model_train(model, model_name, kfold=5):
    oof_preds = np.zeros((train_data.shape[0]))
    test_preds = np.zeros(test_data.shape[0])
    skf = StratifiedKFold(n_splits=kfold, shuffle=True)

    for k, (train_index, test_index) in enumerate(skf.split(train, label)):
        x_train, x_test = train.iloc[train_index, :], train.iloc[test_index, :]
        y_train, y_test = label.iloc[train_index], label.iloc[test_index]

        model.fit(x_train,y_train)

        y_pred = model.predict_proba(x_test)[:,1]
        oof_preds[test_index] = y_pred.ravel()
        auc = roc_auc_score(y_test,y_pred)
        print("Model = %s, KFold = %d, val_auc = %.4f" % (model_name, k, auc))
        test_fold_preds = model.predict_proba(test)[:, 1]
        test_preds += test_fold_preds.ravel()
    print("Overall Model = %s, AUC = %.4f" % (model_name, roc_auc_score(label, oof_preds)))
    return test_preds / kfold

In [22]:
rfc = RandomForestClassifier(n_estimators=100)
rfc_test_preds = model_train(rfc, "RandomForestClassifier", 10)

Model = RandomForestClassifier, KFold = 0, val_auc = 0.8782
Model = RandomForestClassifier, KFold = 1, val_auc = 0.8669
Model = RandomForestClassifier, KFold = 2, val_auc = 0.8721
Model = RandomForestClassifier, KFold = 3, val_auc = 0.8759
Model = RandomForestClassifier, KFold = 4, val_auc = 0.8676
Model = RandomForestClassifier, KFold = 5, val_auc = 0.8782
Model = RandomForestClassifier, KFold = 6, val_auc = 0.8715
Model = RandomForestClassifier, KFold = 7, val_auc = 0.8682
Model = RandomForestClassifier, KFold = 8, val_auc = 0.8691
Model = RandomForestClassifier, KFold = 9, val_auc = 0.8708
Overall Model = RandomForestClassifier, AUC = 0.8718


In [23]:
etc = ExtraTreesClassifier(n_estimators=150)
etc_test_preds = model_train(etc, "ExtraTreesClassifier", 10)

Model = ExtraTreesClassifier, KFold = 0, val_auc = 0.8708
Model = ExtraTreesClassifier, KFold = 1, val_auc = 0.8694
Model = ExtraTreesClassifier, KFold = 2, val_auc = 0.8633
Model = ExtraTreesClassifier, KFold = 3, val_auc = 0.8672
Model = ExtraTreesClassifier, KFold = 4, val_auc = 0.8598
Model = ExtraTreesClassifier, KFold = 5, val_auc = 0.8643
Model = ExtraTreesClassifier, KFold = 6, val_auc = 0.8662
Model = ExtraTreesClassifier, KFold = 7, val_auc = 0.8690
Model = ExtraTreesClassifier, KFold = 8, val_auc = 0.8571
Model = ExtraTreesClassifier, KFold = 9, val_auc = 0.8652
Overall Model = ExtraTreesClassifier, AUC = 0.8652


In [24]:
gbc = GradientBoostingClassifier()
gbc_test_preds = model_train(gbc, "GradientBoostingClassifier", 10)

Model = GradientBoostingClassifier, KFold = 0, val_auc = 0.8772
Model = GradientBoostingClassifier, KFold = 1, val_auc = 0.8755
Model = GradientBoostingClassifier, KFold = 2, val_auc = 0.8761
Model = GradientBoostingClassifier, KFold = 3, val_auc = 0.8721
Model = GradientBoostingClassifier, KFold = 4, val_auc = 0.8727
Model = GradientBoostingClassifier, KFold = 5, val_auc = 0.8671
Model = GradientBoostingClassifier, KFold = 6, val_auc = 0.8730
Model = GradientBoostingClassifier, KFold = 7, val_auc = 0.8684
Model = GradientBoostingClassifier, KFold = 8, val_auc = 0.8811
Model = GradientBoostingClassifier, KFold = 9, val_auc = 0.8787
Overall Model = GradientBoostingClassifier, AUC = 0.8741


In [25]:
hgbc = HistGradientBoostingClassifier()
hgbc_test_preds = model_train(hgbc, "HistGradientBoostingClassifier", 10)

Model = HistGradientBoostingClassifier, KFold = 0, val_auc = 0.8825
Model = HistGradientBoostingClassifier, KFold = 1, val_auc = 0.8812
Model = HistGradientBoostingClassifier, KFold = 2, val_auc = 0.8880
Model = HistGradientBoostingClassifier, KFold = 3, val_auc = 0.8751
Model = HistGradientBoostingClassifier, KFold = 4, val_auc = 0.8748
Model = HistGradientBoostingClassifier, KFold = 5, val_auc = 0.8853
Model = HistGradientBoostingClassifier, KFold = 6, val_auc = 0.8849
Model = HistGradientBoostingClassifier, KFold = 7, val_auc = 0.8957
Model = HistGradientBoostingClassifier, KFold = 8, val_auc = 0.8860
Model = HistGradientBoostingClassifier, KFold = 9, val_auc = 0.8797
Overall Model = HistGradientBoostingClassifier, AUC = 0.8833


In [26]:
xgbc = XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    n_estimators=100, 
    max_depth=10, 
    learning_rate=0.1
)
xgbc_test_preds = model_train(xgbc, "XGBClassifier", 10)

Model = XGBClassifier, KFold = 0, val_auc = 0.8878
Model = XGBClassifier, KFold = 1, val_auc = 0.8719
Model = XGBClassifier, KFold = 2, val_auc = 0.8807
Model = XGBClassifier, KFold = 3, val_auc = 0.8824
Model = XGBClassifier, KFold = 4, val_auc = 0.8810
Model = XGBClassifier, KFold = 5, val_auc = 0.8841
Model = XGBClassifier, KFold = 6, val_auc = 0.8777
Model = XGBClassifier, KFold = 7, val_auc = 0.8769
Model = XGBClassifier, KFold = 8, val_auc = 0.8825
Model = XGBClassifier, KFold = 9, val_auc = 0.8792
Overall Model = XGBClassifier, AUC = 0.8803


In [27]:
gbm = LGBMClassifier(
    objective='binary',
    num_leaves=15, 
    learning_rate=0.1, 
    n_estimators=100, 
    metrics='auc'
)
gbm_test_preds = model_train(gbm, "LGBMClassifier", 10)

Model = LGBMClassifier, KFold = 0, val_auc = 0.8832
Model = LGBMClassifier, KFold = 1, val_auc = 0.8824
Model = LGBMClassifier, KFold = 2, val_auc = 0.8719
Model = LGBMClassifier, KFold = 3, val_auc = 0.8828
Model = LGBMClassifier, KFold = 4, val_auc = 0.8795
Model = LGBMClassifier, KFold = 5, val_auc = 0.8924
Model = LGBMClassifier, KFold = 6, val_auc = 0.8883
Model = LGBMClassifier, KFold = 7, val_auc = 0.8771
Model = LGBMClassifier, KFold = 8, val_auc = 0.8732
Model = LGBMClassifier, KFold = 9, val_auc = 0.8834
Overall Model = LGBMClassifier, AUC = 0.8814


In [28]:
cbc = CatBoostClassifier(iterations=200, depth=10, learning_rate=0.1, loss_function='Logloss', logging_level='Verbose')
cbc_test_preds = model_train(cbc, "CatBoostClassifier", 10)

0:	learn: 0.6155641	total: 267ms	remaining: 53.1s
1:	learn: 0.5574147	total: 332ms	remaining: 32.9s
2:	learn: 0.5194639	total: 395ms	remaining: 26s
3:	learn: 0.4898276	total: 459ms	remaining: 22.5s
4:	learn: 0.4615494	total: 526ms	remaining: 20.5s
5:	learn: 0.4418810	total: 592ms	remaining: 19.1s
6:	learn: 0.4288052	total: 655ms	remaining: 18.1s
7:	learn: 0.4149628	total: 720ms	remaining: 17.3s
8:	learn: 0.4058326	total: 785ms	remaining: 16.7s
9:	learn: 0.3973066	total: 853ms	remaining: 16.2s
10:	learn: 0.3895627	total: 918ms	remaining: 15.8s
11:	learn: 0.3840709	total: 984ms	remaining: 15.4s
12:	learn: 0.3781043	total: 1.05s	remaining: 15.1s
13:	learn: 0.3739309	total: 1.11s	remaining: 14.8s
14:	learn: 0.3710124	total: 1.18s	remaining: 14.5s
15:	learn: 0.3668682	total: 1.25s	remaining: 14.4s
16:	learn: 0.3640539	total: 1.31s	remaining: 14.1s
17:	learn: 0.3620819	total: 1.38s	remaining: 14s
18:	learn: 0.3595225	total: 1.45s	remaining: 13.8s
19:	learn: 0.3582019	total: 1.51s	remaining: 

162:	learn: 0.2823248	total: 11.1s	remaining: 2.53s
163:	learn: 0.2820236	total: 11.2s	remaining: 2.46s
164:	learn: 0.2815523	total: 11.3s	remaining: 2.39s
165:	learn: 0.2813089	total: 11.3s	remaining: 2.32s
166:	learn: 0.2806987	total: 11.4s	remaining: 2.25s
167:	learn: 0.2803217	total: 11.5s	remaining: 2.18s
168:	learn: 0.2797872	total: 11.5s	remaining: 2.12s
169:	learn: 0.2797267	total: 11.6s	remaining: 2.05s
170:	learn: 0.2789500	total: 11.7s	remaining: 1.98s
171:	learn: 0.2784654	total: 11.7s	remaining: 1.91s
172:	learn: 0.2780070	total: 11.8s	remaining: 1.84s
173:	learn: 0.2777874	total: 11.9s	remaining: 1.77s
174:	learn: 0.2774796	total: 11.9s	remaining: 1.71s
175:	learn: 0.2768985	total: 12s	remaining: 1.64s
176:	learn: 0.2764064	total: 12.1s	remaining: 1.57s
177:	learn: 0.2759759	total: 12.1s	remaining: 1.5s
178:	learn: 0.2756248	total: 12.2s	remaining: 1.43s
179:	learn: 0.2751648	total: 12.3s	remaining: 1.36s
180:	learn: 0.2748075	total: 12.3s	remaining: 1.29s
181:	learn: 0.2

123:	learn: 0.3024919	total: 8.41s	remaining: 5.16s
124:	learn: 0.3020279	total: 8.48s	remaining: 5.09s
125:	learn: 0.3015758	total: 8.55s	remaining: 5.02s
126:	learn: 0.3012281	total: 8.62s	remaining: 4.95s
127:	learn: 0.3007815	total: 8.69s	remaining: 4.89s
128:	learn: 0.3001285	total: 8.76s	remaining: 4.82s
129:	learn: 0.2998423	total: 8.82s	remaining: 4.75s
130:	learn: 0.2997196	total: 8.89s	remaining: 4.68s
131:	learn: 0.2993641	total: 8.96s	remaining: 4.61s
132:	learn: 0.2985799	total: 9.02s	remaining: 4.54s
133:	learn: 0.2980083	total: 9.09s	remaining: 4.48s
134:	learn: 0.2971804	total: 9.17s	remaining: 4.41s
135:	learn: 0.2969720	total: 9.23s	remaining: 4.34s
136:	learn: 0.2965099	total: 9.3s	remaining: 4.28s
137:	learn: 0.2962400	total: 9.36s	remaining: 4.21s
138:	learn: 0.2958598	total: 9.43s	remaining: 4.14s
139:	learn: 0.2952385	total: 9.49s	remaining: 4.07s
140:	learn: 0.2947791	total: 9.56s	remaining: 4s
141:	learn: 0.2943490	total: 9.63s	remaining: 3.93s
142:	learn: 0.29

84:	learn: 0.3173038	total: 5.87s	remaining: 7.94s
85:	learn: 0.3171279	total: 5.94s	remaining: 7.87s
86:	learn: 0.3169542	total: 6s	remaining: 7.8s
87:	learn: 0.3167138	total: 6.07s	remaining: 7.73s
88:	learn: 0.3162749	total: 6.14s	remaining: 7.66s
89:	learn: 0.3161297	total: 6.2s	remaining: 7.58s
90:	learn: 0.3156562	total: 6.27s	remaining: 7.51s
91:	learn: 0.3143126	total: 6.35s	remaining: 7.45s
92:	learn: 0.3138331	total: 6.42s	remaining: 7.39s
93:	learn: 0.3135557	total: 6.49s	remaining: 7.32s
94:	learn: 0.3132919	total: 6.56s	remaining: 7.25s
95:	learn: 0.3127662	total: 6.63s	remaining: 7.18s
96:	learn: 0.3123177	total: 6.7s	remaining: 7.11s
97:	learn: 0.3122072	total: 6.77s	remaining: 7.04s
98:	learn: 0.3119242	total: 6.84s	remaining: 6.98s
99:	learn: 0.3115624	total: 6.91s	remaining: 6.91s
100:	learn: 0.3113463	total: 6.98s	remaining: 6.84s
101:	learn: 0.3109190	total: 7.05s	remaining: 6.78s
102:	learn: 0.3098605	total: 7.12s	remaining: 6.71s
103:	learn: 0.3094445	total: 7.19s

46:	learn: 0.3345849	total: 3.23s	remaining: 10.5s
47:	learn: 0.3341844	total: 3.3s	remaining: 10.4s
48:	learn: 0.3331587	total: 3.37s	remaining: 10.4s
49:	learn: 0.3322187	total: 3.43s	remaining: 10.3s
50:	learn: 0.3318550	total: 3.5s	remaining: 10.2s
51:	learn: 0.3312923	total: 3.57s	remaining: 10.2s
52:	learn: 0.3307974	total: 3.64s	remaining: 10.1s
53:	learn: 0.3301551	total: 3.71s	remaining: 10s
54:	learn: 0.3299195	total: 3.78s	remaining: 9.97s
55:	learn: 0.3293236	total: 3.85s	remaining: 9.9s
56:	learn: 0.3287705	total: 3.92s	remaining: 9.82s
57:	learn: 0.3284541	total: 3.98s	remaining: 9.75s
58:	learn: 0.3280578	total: 4.05s	remaining: 9.67s
59:	learn: 0.3271330	total: 4.12s	remaining: 9.62s
60:	learn: 0.3264155	total: 4.19s	remaining: 9.55s
61:	learn: 0.3254930	total: 4.26s	remaining: 9.49s
62:	learn: 0.3251786	total: 4.33s	remaining: 9.43s
63:	learn: 0.3247051	total: 4.4s	remaining: 9.36s
64:	learn: 0.3241313	total: 4.47s	remaining: 9.28s
65:	learn: 0.3234913	total: 4.54s	rem

...

92:	learn: 0.3150914	total: 6.22s	remaining: 7.16s
93:	learn: 0.3148229	total: 6.28s	remaining: 7.08s
94:	learn: 0.3146403	total: 6.35s	remaining: 7.01s
95:	learn: 0.3139653	total: 6.42s	remaining: 6.95s
96:	learn: 0.3133424	total: 6.48s	remaining: 6.88s
97:	learn: 0.3129926	total: 6.55s	remaining: 6.82s
98:	learn: 0.3126156	total: 6.62s	remaining: 6.75s
99:	learn: 0.3122007	total: 6.68s	remaining: 6.68s
100:	learn: 0.3114154	total: 6.75s	remaining: 6.62s
101:	learn: 0.3111316	total: 6.82s	remaining: 6.55s
102:	learn: 0.3105870	total: 6.88s	remaining: 6.48s
103:	learn: 0.3103566	total: 6.95s	remaining: 6.42s
104:	learn: 0.3099636	total: 7.02s	remaining: 6.35s
105:	learn: 0.3097318	total: 7.09s	remaining: 6.28s
106:	learn: 0.3085603	total: 7.16s	remaining: 6.22s
107:	learn: 0.3082431	total: 7.22s	remaining: 6.15s
108:	learn: 0.3077963	total: 7.29s	remaining: 6.08s
109:	learn: 0.3071893	total: 7.35s	remaining: 6.02s
110:	learn: 0.3068697	total: 7.42s	remaining: 5.95s
111:	learn: 0.306503

52:	learn: 0.3316672	total: 3.56s	remaining: 9.88s
53:	learn: 0.3313515	total: 3.63s	remaining: 9.81s
54:	learn: 0.3309352	total: 3.69s	remaining: 9.74s
55:	learn: 0.3306841	total: 3.76s	remaining: 9.67s
56:	learn: 0.3304527	total: 3.83s	remaining: 9.6s
57:	learn: 0.3293447	total: 3.9s	remaining: 9.54s
58:	learn: 0.3290886	total: 3.96s	remaining: 9.47s
59:	learn: 0.3287898	total: 4.03s	remaining: 9.4s
60:	learn: 0.3279841	total: 4.1s	remaining: 9.34s
61:	learn: 0.3278064	total: 4.16s	remaining: 9.26s
62:	learn: 0.3274708	total: 4.23s	remaining: 9.2s
63:	learn: 0.3268338	total: 4.3s	remaining: 9.13s
64:	learn: 0.3264777	total: 4.37s	remaining: 9.07s
65:	learn: 0.3258859	total: 4.43s	remaining: 9s
66:	learn: 0.3255469	total: 4.5s	remaining: 8.93s
67:	learn: 0.3251393	total: 4.57s	remaining: 8.87s
68:	learn: 0.3245771	total: 4.63s	remaining: 8.8s
69:	learn: 0.3242703	total: 4.7s	remaining: 8.73s
70:	learn: 0.3239215	total: 4.76s	remaining: 8.66s
71:	learn: 0.3237438	total: 4.83s	remaining

11:	learn: 0.3818543	total: 795ms	remaining: 12.5s
12:	learn: 0.3765769	total: 861ms	remaining: 12.4s
13:	learn: 0.3724154	total: 928ms	remaining: 12.3s
14:	learn: 0.3686560	total: 996ms	remaining: 12.3s
15:	learn: 0.3657968	total: 1.06s	remaining: 12.2s
16:	learn: 0.3633310	total: 1.13s	remaining: 12.2s
17:	learn: 0.3611729	total: 1.2s	remaining: 12.1s
18:	learn: 0.3590291	total: 1.26s	remaining: 12s
19:	learn: 0.3573893	total: 1.33s	remaining: 11.9s
20:	learn: 0.3564416	total: 1.4s	remaining: 11.9s
21:	learn: 0.3548677	total: 1.46s	remaining: 11.8s
22:	learn: 0.3524482	total: 1.53s	remaining: 11.8s
23:	learn: 0.3515515	total: 1.59s	remaining: 11.7s
24:	learn: 0.3492740	total: 1.66s	remaining: 11.6s
25:	learn: 0.3481878	total: 1.73s	remaining: 11.6s
26:	learn: 0.3472155	total: 1.79s	remaining: 11.5s
27:	learn: 0.3460368	total: 1.86s	remaining: 11.4s
28:	learn: 0.3456003	total: 1.92s	remaining: 11.3s
29:	learn: 0.3445493	total: 1.99s	remaining: 11.3s
30:	learn: 0.3434680	total: 2.05s	r

174:	learn: 0.2757146	total: 11.6s	remaining: 1.66s
175:	learn: 0.2753719	total: 11.7s	remaining: 1.6s
176:	learn: 0.2751062	total: 11.8s	remaining: 1.53s
177:	learn: 0.2743198	total: 11.8s	remaining: 1.46s
178:	learn: 0.2735523	total: 11.9s	remaining: 1.4s
179:	learn: 0.2731632	total: 12s	remaining: 1.33s
180:	learn: 0.2728488	total: 12s	remaining: 1.26s
181:	learn: 0.2725046	total: 12.1s	remaining: 1.2s
182:	learn: 0.2722243	total: 12.2s	remaining: 1.13s
183:	learn: 0.2720924	total: 12.2s	remaining: 1.06s
184:	learn: 0.2717910	total: 12.3s	remaining: 998ms
185:	learn: 0.2717615	total: 12.4s	remaining: 932ms
186:	learn: 0.2711629	total: 12.4s	remaining: 865ms
187:	learn: 0.2711427	total: 12.5s	remaining: 798ms
188:	learn: 0.2705803	total: 12.6s	remaining: 732ms
189:	learn: 0.2700858	total: 12.6s	remaining: 665ms
190:	learn: 0.2695458	total: 12.7s	remaining: 599ms
191:	learn: 0.2690386	total: 12.8s	remaining: 532ms
192:	learn: 0.2688737	total: 12.8s	remaining: 466ms
193:	learn: 0.26861

135:	learn: 0.2952846	total: 9.04s	remaining: 4.26s
136:	learn: 0.2950055	total: 9.11s	remaining: 4.19s
137:	learn: 0.2945794	total: 9.18s	remaining: 4.12s
138:	learn: 0.2940249	total: 9.24s	remaining: 4.06s
139:	learn: 0.2934921	total: 9.31s	remaining: 3.99s
140:	learn: 0.2931005	total: 9.37s	remaining: 3.92s
141:	learn: 0.2928231	total: 9.44s	remaining: 3.85s
142:	learn: 0.2926337	total: 9.51s	remaining: 3.79s
143:	learn: 0.2922401	total: 9.57s	remaining: 3.72s
144:	learn: 0.2922115	total: 9.64s	remaining: 3.65s
145:	learn: 0.2917744	total: 9.71s	remaining: 3.59s
146:	learn: 0.2913437	total: 9.77s	remaining: 3.52s
147:	learn: 0.2908656	total: 9.84s	remaining: 3.46s
148:	learn: 0.2903073	total: 9.91s	remaining: 3.39s
149:	learn: 0.2896441	total: 9.97s	remaining: 3.33s
150:	learn: 0.2890951	total: 10s	remaining: 3.26s
151:	learn: 0.2885381	total: 10.1s	remaining: 3.19s
152:	learn: 0.2880294	total: 10.2s	remaining: 3.13s
153:	learn: 0.2877857	total: 10.2s	remaining: 3.06s
154:	learn: 0.

In [29]:
preds = (cbc_test_preds + gbm_test_preds + xgbc_test_preds + hgbc_test_preds + gbc_test_preds + etc_test_preds) / 6

In [30]:
submission['label'] = preds

In [31]:
submission.to_csv('submission.csv', index=False)